In [84]:
import boto3
from time import gmtime, strftime

sm_client = boto3.client('sagemaker')

In [85]:
# Create Sagemaker Model

model_name = 'bedrock-x' + '-roop-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
container = '891376917605.dkr.ecr.us-west-2.amazonaws.com/bedrock-x-roop:latest'
instance_type = 'ml.g4dn.xlarge'

print('Model name: ' + model_name)
print('Container image: ' + container)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': container,
        'Environment': {}
    },
    ExecutionRoleArn='arn:aws:iam::891376917605:role/sm-excution-role-bedrock-x'
)

print("Model Arn: " + create_model_response['ModelArn'])

Model name: bedrock-x-roop-2024-06-20-05-36-48
Container image: 891376917605.dkr.ecr.us-west-2.amazonaws.com/bedrock-x:latest
Model Arn: arn:aws:sagemaker:us-west-2:891376917605:model/bedrock-x-roop-2024-06-20-05-36-48


In [86]:
# Create Sagemaker Endpoint Config
endpoint_config_name = 'bedrock-x' + '-roop-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name, 
    ProductionVariants = [{
        'ModelName': model_name,
        'VariantName': 'AllTraffic',
        'InitialInstanceCount': 1,
        'InstanceType': instance_type
    }])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: bedrock-x-roop-2024-06-20-05-36-50
Endpoint config Arn: arn:aws:sagemaker:us-west-2:891376917605:endpoint-config/bedrock-x-roop-2024-06-20-05-36-50


In [87]:
endpoint_name = 'bedrock-x' + '-roop-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: bedrock-x-roop-2024-06-20-05-36-53
Endpoint Arn: arn:aws:sagemaker:us-west-2:891376917605:endpoint/bedrock-x-roop-2024-06-20-05-36-53
Endpoint Status: Creating
Waiting for bedrock-x-roop-2024-06-20-05-36-53 endpoint to be in service...


In [88]:
import json

runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

content_type = "application/json"
request_body = {
    "uuid": "jungseob", 
    "bucket": "bedrock-x-sm-shared", 
    "source": "images/source/source.png", 
    "target": "images/target/target.png", 
    "output": "images/output/output.png"
}

payload = json.dumps(request_body)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())
result

{'bucket': 'bedrock-x-sm-shared',
 'output': 'images/output/output.png',
 'source': 'images/source/source.png',
 'target': 'images/target/target.png',
 'uuid': 'jungseob'}